In [27]:
import pandas as pd 
# import requests
from dotenv import load_dotenv
import os 
import base64
from requests import post, get
import json
import psycopg2

In [28]:
#load the environment variables
load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

#Test they have been loaded correctly
# print(client_id, client_secret)

In [29]:
# Uses client credentials to request access token
def get_token():
    auth_info = client_id + ":" + client_secret
    auth_bytes = auth_info.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    
    base_url = "https://accounts.spotify.com/api/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Authorization": "Basic " + auth_base64
    }
    data = {"grant_type": "client_credentials"}
    result = post(base_url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

token = get_token()
print(token)
    

BQC5iwEIYhYcqB9oNI6G1pXJVlKZvnwL0utaUY-ChtcYjn2RNsF5_j6QELrX0lZu9cvdA9i0KVlbkT4Z87QBiktXy2k2P7qOCPk6wsHCHB7LTFLXmqY


In [30]:
#Forms header for whenever we send requests
def get_header(token):
    return {"Authorization": "Bearer " + token}

In [31]:
def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"
    
    query_url = url + query
    
    result = get(query_url, headers=headers)
    # print(f"Raw response for {artist_name}: {result.content}")  # Debugging line
    try:
        json_result = json.loads(result.content)
    except Exception as e:
        print(f"Error parsing JSON for {artist_name}: {e}")
        return None

    # Check for errors in the JSON response
    if "error" in json_result:
        print(f"API returned an error for {artist_name}: {json_result['error']}")
        return None
    
    # Extract the "artists" key
    json_result = json_result.get("artists", {}).get("items", [])
    
    if len(json_result) == 0:
        print(f"No artist with this name exists: {artist_name}")
        return None
    
    return json_result[0]
 

In [41]:
def get_songs_by_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
    headers = get_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)["tracks"]
    return json_result

In [55]:
token = get_token()
result = search_for_artist(token, "f")

# print(result)
artist_id = result["id"]

print(artist_id)
# print(result["name"])
songs = get_songs_by_artist(token, artist_id)
print(songs)



# for idx, song in enumerate(songs):
#     print(f"{idx + 1}. {song['name']}")

2gJLanGIV9JqQuKBCWy9ZG
[{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2gJLanGIV9JqQuKBCWy9ZG'}, 'href': 'https://api.spotify.com/v1/artists/2gJLanGIV9JqQuKBCWy9ZG', 'id': '2gJLanGIV9JqQuKBCWy9ZG', 'name': 'Peppa Pig', 'type': 'artist', 'uri': 'spotify:artist:2gJLanGIV9JqQuKBCWy9ZG'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT',

In [39]:
# df = pd.DataFrame(songs)
# df2 = pd.DataFrame(result)
df1 = pd.DataFrame(songs)

# Select only the 'id', 'name', and 'popularity' columns
filtered_df = df1[['id', 'name', 'popularity', 'duration_ms']]

# Display the filtered DataFrame
print(filtered_df)

                       id                                name  popularity  \
0  1eTaznNW4Xxtx9za2SMTXB                   DENIAL IS A RIVER          84   
1  2dKkVF2m160z0RNDN2dddc                       NISSAN ALTIMA          79   
2  35D1RA5uVFVhMc1qX80TaQ             Balloon (feat. Doechii)          80   
3  3QQvSQKV8YmQxGolwwWe59                             CATFISH          76   
4  73RbfOTJIjHzi2pcVHjeHM           What It Is (Solo Version)          73   
5  3Ehr6qjkJQTSL1LjsWELQb                      BOILED PEANUTS          72   
6  7EIPVo13rl0uwKbjubMnc7  I'M HIS, HE'S MINE (feat. Doechii)          67   
7  2xaaJcmvMOmpDbTwlWnjFh                 Alter Ego (with JT)          71   
8  27oSDhuYJWxdm9yjI6891v                            BULLFROG          70   
9  3FjK86616FbluOfTxNK2gY       A N X I E T Y (feat. Doechii)          69   

   duration_ms  
0       159586  
1       126832  
2       154261  
3       134097  
4       189824  
5       121858  
6       198461  
7       204094  

In [ ]:
# List of artists
artists = ["Asake","Burna-Boy","Jungle","Kendrick-Lamar", "SZA","Dexta-Daps", "JID","Teddy-Swims", "Bruno-Mars", "Lady-Gaga", "Coldplay", "Taylor-Swift", "Bad-Bunny", "The-Weeknd", "Billie-Eilish", "Ariana-Grande", "Drake", "Rihanna", "Ed-Sheeran", "Sabrina-Carpenter", "Justin-Bieber", "Ariana-Grande", "Eminem", "Kanye-West", "Post-Malone", "BTS", "Travis-Scott", "Doechii", "Whitney Houston"]

# Initialize an empty list to store data for all artists
all_songs_data = []

# Iterate over each artist in the list
for artist_name in artists:
    try:
        # Search for the artist
        result = search_for_artist(token, artist_name)
        if result is None:
            print(f"No data found for artist: {artist_name}")
            continue
        
        # Get the artist ID and name
        artist_id = result["id"]
        artist_name = result["name"]  # This extracts the correct name of the artist
        artist_popularity = result["popularity"]
        
        # Fetch the top tracks for the artist
        songs = get_songs_by_artist(token, artist_id)
        
        # Append relevant data from the songs to the list
        for song in songs:
            all_songs_data.append({
                "id": song["id"],
                "artist_name": artist_name,  # Add the artist's name to each song entry
                "artist_popularity": artist_popularity,
                "song_name": song["name"],
                "song_popularity": song["popularity"],
                "duration_ms": song["duration_ms"]
                
            })
    except Exception as e:
        print(f"An error occurred for artist {artist_name}: {e}")

# Convert the collected data into a DataFrame
final_df = pd.DataFrame(all_songs_data)

# Transform duration_ms into minutes and seconds (duration_min_sec)
final_df["duration_min_sec"] = final_df["duration_ms"].apply(lambda x: f"{x // 60000}:{(x % 60000) // 1000:02d}")

# Drop the original duration_ms column 
final_df.drop(columns=["duration_ms"], inplace=True)

# Display the DataFrame
final_df

,id,artist_name,artist_popularity,song_name,song_popularity,duration_min_sec
0,2j8n2hpZlEMvtrjJ7n0ZIy,Asake,75,Bad Girl (feat. Asake),71,2:54
1,13VXuHw3O8Yt7VwRIDqSo4,Asake,75,MMS,71,3:39
2,5ZtK8XAVnoaGdBXZWCEVCY,Asake,75,Lonely At The Top,69,2:37
3,1qxKva4IzWde7m2jLpZvDU,Asake,75,Active,65,2:52
4,0Q1bMs3xLQiDEeaneehdxv,Asake,75,Remember,68,3:02
...,...,...,...,...,...,...
275,3Ehr6qjkJQTSL1LjsWELQb,Doechii,80,BOILED PEANUTS,72,2:01
276,7EIPVo13rl0uwKbjubMnc7,Doechii,80,"I'M HIS, HE'S MINE (feat. Doechii)",67,3:18
277,2xaaJcmvMOmpDbTwlWnjFh,Doechii,80,Alter Ego (with JT),71,3:24
278,27oSDhuYJWxdm9yjI6891v,Doechii,80,BULLFROG,70,1:34


In [36]:
from sqlalchemy import create_engine

# Load environment variables from the .env file
load_dotenv()

# Database connection parameters
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")

# Create the connection string
connection_string = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"

# Create an SQLAlchemy engine
engine = create_engine(connection_string)

# Write the DataFrame to the database in the 'student' schema
table_name = "ak_spotify"  # Desired table name
final_df.to_sql(
    table_name,
    engine,
    schema="student",
    if_exists="replace",  # Options: 'fail', 'replace', 'append'
    index=False  # Do not write the DataFrame index as a column
)

280

In [37]:
# Query the database and load the results into a DataFrame
query1 = """
        SELECT * FROM student.ak_spotify ORDER BY artist_popularity DESC      
        """
        
query2 = """
        SELECT AVG(duration_ms) as average_song_length_ms 
        FROM student.ak_spotify       
        """
        
query3 = """
        SELECT artist_name, AVG(duration_ms) as average_song_length_ms, AVG(song_popularity) average_song_popularity
        FROM student.ak_spotify
        GROUP BY artist_name, artist_popularity 
        ORDER BY artist_popularity DESC
        
"""
result_df = pd.read_sql(query3, engine)

# Display the DataFrame
result_df


ProgrammingError: (psycopg2.errors.UndefinedColumn) column "duration_ms" does not exist
LINE 2:         SELECT artist_name, AVG(duration_ms) as average_song...
                                        ^

[SQL: 
        SELECT artist_name, AVG(duration_ms) as average_song_length_ms, AVG(song_popularity) average_song_popularity
        FROM student.ak_spotify
        GROUP BY artist_name, artist_popularity 
        ORDER BY artist_popularity DESC
        
]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [42]:
def convert_mil_to_sec(df):
    # Transform duration_ms into minutes and seconds (duration_min_sec)
    df["duration_min_sec"] = df["duration_ms"].apply(lambda x: f"{x // 60000}:{(x % 60000) // 1000:02d}")

    # Drop the original duration_ms column 
    df.drop(columns=["duration_ms"], inplace=True)
    
    return df

In [45]:
def get_artist_info(artist_name):
    token = get_token()
    artist_info = search_for_artist(token, artist_name)
    artist_id = artist_info["id"]
    artist_name = artist_info["name"]
    artist_popularity = artist_info["popularity"]
    song_info = get_songs_by_artist(token, artist_id)
    
    all_info = []
    
    # Iterate through all songs
    for song in song_info:
        all_info.append({
            "artist_id": artist_id,
            "artist_name": artist_name,  
            "artist_popularity": artist_popularity,
            "song_id": song["id"],
            "song_name": song["name"],
            "song_popularity": song["popularity"],
            "duration_ms": song["duration_ms"]
        })
    
    artist_df = pd.DataFrame(all_info)
    all_info_df = convert_mil_to_sec(artist_df)
    return all_info_df


In [53]:
test = get_artist_info("n")

test

,artist_id,artist_name,artist_popularity,song_id,song_name,song_popularity,duration_min_sec
0,2RQXRUsr4IW1f3mKyKsy4B,Noah Kahan,84,0mflMxspEfB0VbI1kyLiAv,Stick Season,86,3:02
1,2RQXRUsr4IW1f3mKyKsy4B,Noah Kahan,84,4nHJcUtNSUVjXRnjdP29Bk,You’re Gonna Go Far,80,4:46
2,2RQXRUsr4IW1f3mKyKsy4B,Noah Kahan,84,4O2rRsoSPb5aN7N3tG6Y3v,Northern Attitude,75,4:27
3,2RQXRUsr4IW1f3mKyKsy4B,Noah Kahan,84,22VHOlVYBqytsrAqV8yXBK,Dial Drunk (with Post Malone),78,3:33
4,2RQXRUsr4IW1f3mKyKsy4B,Noah Kahan,84,7ByxizhA4GgEf7Sxomxhze,All My Love,77,4:11
5,2RQXRUsr4IW1f3mKyKsy4B,Noah Kahan,84,3yn01PcU95PTbiZ3xvop2j,Call Your Mom,76,4:38
6,2RQXRUsr4IW1f3mKyKsy4B,Noah Kahan,84,1fvC3CmNAbsx0V9ULMU8NB,Sarah's Place (feat. Noah Kahan),74,3:34
7,2RQXRUsr4IW1f3mKyKsy4B,Noah Kahan,84,3Z2anmIVG8b1GelyeFQdnP,Maine,74,3:52
8,2RQXRUsr4IW1f3mKyKsy4B,Noah Kahan,84,745JbQYxoYIegHzHTUxETp,"Everywhere, Everything (with Gracie Abrams)",72,4:17
9,2RQXRUsr4IW1f3mKyKsy4B,Noah Kahan,84,07UFnnK3uPIuKv5Rs9TmXl,Orange Juice,73,4:57
